In [ ]:
!pip install -q transformers==4.32.0 deeplake==3.6.19 wandb==0.15.8 accelerate==0.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 21.3 MB/s eta 0:00:

In [ ]:
# In LambdaLabs
# !pip install protobuf===3.20.3

In [ ]:
# %env WANDB_PROJECT=GenAI360

In [ ]:
# !wandb login

# Load the Model

In [ ]:
# We can also use GPTJ here too.
from transformers import AutoConfig

config = AutoConfig.from_pretrained( "gpt2" )

In [ ]:
print( config )

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 50257
}



In [ ]:
from transformers import GPT2LMHeadModel
from accelerate import Accelerator


model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1e6:.1f}M parameters")

GPT-2 size: 124.4M parameters


We can Scale up the model by changing the following parameters.

In [ ]:
# config.n_layer = 32
# config.n_embd = 1600
# config.n_positions = 512
# config.n_ctx = 512
# config.n_head = 32

In [ ]:
# model = GPT2LMHeadModel(config)
# model_size = sum(t.numel() for t in model.parameters())
# print(f"TowardsGPT size: {model_size/1e6:.1f}M parameters")

# Load Dataset from Deep Lake

In [ ]:
import deeplake

In [ ]:
# Connect to the training and testing datasets
ds = deeplake.load('hub://activeloop/openwebtext-train')
ds_val = deeplake.load('hub://activeloop/openwebtext-val')

|

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/openwebtext-train



|

...

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/openwebtext-val



|

hub://activeloop/openwebtext-val loaded successfully.



In [ ]:
# Limit the dataet to roughly 2B tokens
ds = ds[0:2_000_000]

In [ ]:
# print( ds[0].text.text() )

In [ ]:
# ds.visualize()

# Define Data Loader Class

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, ds):
        self.ds = ds

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        tokenized_text = tokenizer(
            self.ds.text[idx].text(),
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors="pt"
        )
        tokenized_text = tokenized_text['input_ids'][0]

        # Trainer will take care of shifting the labels.
        sample = {"input_ids": tokenized_text, "labels": tokenized_text}

        return sample

In [ ]:
myTrainingLoader = MyDataset(ds)
myValidationLoader = MyDataset(ds_val)

# Training

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
args = TrainingArguments(
    output_dir="GPT2-scratch-openwebtext",
    evaluation_strategy="epoch",
    save_strategy="steps",
    save_steps=1000,
    num_train_epochs=10,
    logging_steps=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    fp16=True,
    ddp_find_unused_parameters=False,
    run_name="GPT2-scratch-openwebtext",
    report_to="wandb"
)

In [ ]:
class TrainerWithDataLoaders(Trainer):
    def __init__(self, *args, train_dataloader=None, eval_dataloader=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.train_dataloader = train_dataloader
        self.eval_dataloader = eval_dataloader

    def get_train_dataloader(self):
        return self.train_dataloader

    def get_eval_dataloader(self, dummy):
        return self.eval_dataloader

In [ ]:
trainer = Trainer( # TrainerWithDataLoaders()
    model=model,
    args=args,
    train_dataset=myTrainingLoader,
    eval_dataset=myValidationLoader,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

# Inference

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation",
                model="./mode",
                tokenizer=tokenizer,
                device=device)

In [ ]:
txt = "The house prices dropped down"

completion = pipe(txt, num_return_sequences=1)
print( completion )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The house prices dropped down to 3.02% last year. While it was still in development, the housing market was still down. The recession hit on 3 years between 1998 and 2011. In fact it slowed the amount of housing from 2013 to 2013'}]


In [ ]:
print( completion[0]['generated_text'] )

The house prices dropped down to 3.02% last year. While it was still in development, the housing market was still down. The recession hit on 3 years between 1998 and 2011. In fact it slowed the amount of housing from 2013 to 2013
